In [1]:
import opal
import kglab
import rdflib
import z3
import dateutil.parser
import re
import numpy as np
import pandas as pd
import os
import sexpdata

In [2]:
from opal.logic.z3.z3_literal import Z3Literal
from opal.data.event_data import EventData
from opal.data.mapper import LogMapper
from opal.data.processor import LogProcessor

import opal.logic.z3.z3_literal as z3_literal
from opal.logic.z3.z3_ontology import Z3Ontology as Ontology
from opal.logic.z3.z3_reasoner import Z3Reasoner as Reasoner
from opal.logic.z3.z3_ontology import REF_ONTOLOGY_Z3, PSL_CORE_Z3, PSL_COMPLEX_Z3, PSL_OCCTREE_Z3, PSL_SUBACTIVITY_Z3, PSL_OCC_PREC

from rdflib import Graph

from z3 import *
z3.set_param(proof=True)
z3.set_param('verbose', 0)
z3.set_option('model.compact', True)

Updated ontology environment with 1 sorts and 13 functions.
Adding axiom: Reference Taxonomy_ont_t_0_disjoint_classes
Adding axiom: Reference Taxonomy_ont_t_1_event_order_SIMPLE
Adding axiom: Reference Taxonomy_ont_t_2_infer_next_event
Updated ontology environment with 1 sorts and 26 functions.
Adding axiom: PSL Core Ontology_ont_t_0_type_disjointness
Adding axiom: PSL Core Ontology_ont_t_1_begin_unique
Adding axiom: PSL Core Ontology_ont_t_2_ends_unique
Adding axiom: PSL Core Ontology_ont_t_3_occurrence_bounds
Adding axiom: PSL Core Ontology_ont_t_4_occurrence_sort_constraints
Adding axiom: PSL Core Ontology_ont_t_5_unique_activity_occurrence
Adding axiom: PSL Core Ontology_ont_t_6_occurrence_has_activity
Adding axiom: PSL Core Ontology_ont_t_7_participation_t_sort_constraints
Adding axiom: PSL Core Ontology_ont_t_8_participation_time_to_generic
Adding axiom: PSL Core Ontology_ont_t_9_participation_sort_constraints
Adding axiom: PSL Core Ontology_ont_t_10_participation_temporal_constr

In [3]:
data_dir = os.path.abspath('../data/')
ttl_data_file = data_dir + '/P1_log_instances.ttl'
sample_log_file = data_dir + '/sample_log.csv'
bpic2017_file = data_dir + '/BPI Challenge 2017.xes.gz'

ontology_dir = os.path.abspath('../../opal/logic/z3/ontologies/')
psl_core_file = ontology_dir + '/PSL/PSL_core.smt2'
psl_core_mapping_file = ontology_dir + '/PSL/PSL_core_mapping.smt2'

## Mapping and reasoning on a simple XES log

### Mapping

In [4]:
# load the log
col_dict = {'caseID' : 'caseID', 'activityID' : 'activityID', 'timestamp' : 'timestamp', 'eventID' : None, 'transition' : 'transition'}
processor = LogProcessor(col_dict=col_dict, log_path=sample_log_file, batch_size=1)
event_data = processor.load_log()

In [5]:
# viewing the original data
event_data.data

,caseID,activityID,eventID,timestamp,resourceID,transition,processID,batch
0,C_case_0,A_activity_A,E_0,2016-01-01 09:00:00+00:00,user_1,T_start,P1,1
1,C_case_0,A_activity_A,E_1,2016-01-01 09:15:00+00:00,user_1,T_complete,P1,1
2,C_case_0,A_activity_C,E_2,2016-01-01 09:35:00+00:00,user_2,T_complete,P1,1
3,C_case_0,A_activity_C,E_3,2016-01-01 09:20:00+00:00,user_2,T_start,P1,1
4,C_case_1,A_activity_A,E_4,NaT,user_1,T_complete,P1,2
5,C_case_1,A_activity_B,E_5,2016-01-02 08:50:00+00:00,user_0,T_start,P1,2
6,C_case_1,A_activity_B,E_6,2016-01-02 09:00:00+00:00,user_0,T_complete,P1,2
7,C_case_1,A_activity_C,E_7,2016-01-03 09:00:00+00:00,user_0,T_complete,P1,2
8,C_case_2,A_activity_A,E_8,2016-01-01 09:00:00+00:00,user_1,T_complete,P1,3
9,C_case_2,A_activity_B,E_9,2016-01-01 09:10:00+00:00,user_1,T_complete,P1,3


In [6]:
# metadata is automatically created
event_data.metadata

{'log_paths': ['/tmp/tmpdz0bz_1t.csv',
  '/tmp/tmpq5xb47rj.csv',
  '/tmp/tmp7_ar_0h8.csv'],
 'process_name': 'P1',
 'batches': 3}

In [7]:
# initialize a mapper to map the log to logical facts according to the specified encoding (here, z3)
mapper = LogMapper(data=event_data, output_encoding='z3')
z3_data = mapper.get_mapped_data()

Building mappings for logs...


  0%|          | 0/3 [00:00<?, ?it/s]2025-11-25 22:17:15,762 | INFO: Translating YARRRML mapping to [R2]RML
2025-11-25 22:17:15,764 | INFO: RML content is created!
2025-11-25 22:17:15,773 | INFO: Mapping has been syntactically validated.
2025-11-25 22:17:15,774 | INFO: Translation has finished successfully.
2025-11-25 22:17:15,787 | INFO: Translating YARRRML mapping to [R2]RML
2025-11-25 22:17:15,789 | INFO: RML content is created!
2025-11-25 22:17:15,799 | INFO: Mapping has been syntactically validated.
2025-11-25 22:17:15,801 | INFO: Translation has finished successfully.
2025-11-25 22:17:15,809 | INFO: Translating YARRRML mapping to [R2]RML
2025-11-25 22:17:15,810 | INFO: RML content is created!
2025-11-25 22:17:15,817 | INFO: Mapping has been syntactically validated.
2025-11-25 22:17:15,818 | INFO: Translation has finished successfully.
100%|██████████| 3/3 [00:00<00:00, 42.24it/s]

Mappings built.
Generating knowledge graph...



INFO | 2025-11-25 22:17:16,490 | 11 mapping rules retrieved.
INFO | 2025-11-25 22:17:16,506 | Mapping partition with 11 groups generated.
INFO | 2025-11-25 22:17:16,508 | Maximum number of rules within mapping group: 1.
INFO | 2025-11-25 22:17:16,508 | Mappings processed in 0.683 seconds.
INFO | 2025-11-25 22:17:16,980 | Number of triples generated in total: 32.


Knowledge graph generated.
Knowledge graph 1/3 generated.
Generating knowledge graph...


INFO | 2025-11-25 22:17:17,719 | 11 mapping rules retrieved.
INFO | 2025-11-25 22:17:17,736 | Mapping partition with 11 groups generated.
INFO | 2025-11-25 22:17:17,738 | Maximum number of rules within mapping group: 1.
INFO | 2025-11-25 22:17:17,740 | Mappings processed in 0.741 seconds.
INFO | 2025-11-25 22:17:18,199 | Number of triples generated in total: 32.


Knowledge graph generated.
Knowledge graph 2/3 generated.
Generating knowledge graph...


INFO | 2025-11-25 22:17:18,959 | 11 mapping rules retrieved.
INFO | 2025-11-25 22:17:18,978 | Mapping partition with 11 groups generated.
INFO | 2025-11-25 22:17:18,980 | Maximum number of rules within mapping group: 1.
INFO | 2025-11-25 22:17:18,980 | Mappings processed in 0.771 seconds.
INFO | 2025-11-25 22:17:19,407 | Number of triples generated in total: 33.


Knowledge graph generated.
Knowledge graph 3/3 generated.
Z3 literal set (1/3) generated.
Z3 literal set (2/3) generated.
Z3 literal set (3/3) generated.


In [8]:
# rdf is still generated, because it is a prerequisite for mapping to more expressive encoding languages
graph = mapper.kgs[0].rdf_graph()
for triple in graph.triples((None, None, None)):
    print(triple)

(rdflib.term.URIRef('http://www.example.com/E_3'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('https://stl.mie.utoronto.ca/ontologies/spm/Event'))
(rdflib.term.URIRef('http://www.example.com/E_0'), rdflib.term.URIRef('https://stl.mie.utoronto.ca/ontologies/spm/hasResource'), rdflib.term.URIRef('http://www.example.com/user_1'))
(rdflib.term.URIRef('http://www.example.com/E_3'), rdflib.term.URIRef('https://stl.mie.utoronto.ca/ontologies/spm/hasRecordedTime'), rdflib.term.Literal('2016-01-01 09:20:00+00:00', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#dateTimeStamp')))
(rdflib.term.URIRef('http://www.example.com/E_2'), rdflib.term.URIRef('https://stl.mie.utoronto.ca/ontologies/spm/hasLifecycleTransition'), rdflib.term.URIRef('http://www.example.com/T_complete'))
(rdflib.term.URIRef('http://www.example.com/E_3'), rdflib.term.URIRef('https://stl.mie.utoronto.ca/ontologies/spm/hasLifecycleTransition'), rdflib.term.URIRef('http:

### Reasoning Capabilities

In [9]:
# load the ontology (together with the mapping)
psl_core = Ontology.from_smt2(psl_core_file, psl_core_mapping_file)

Updated ontology environment with 1 sorts and 25 functions.
Adding axiom: None_ont_t_0_type_disjointness
Adding axiom: None_ont_t_1_begin_unique
Adding axiom: None_ont_t_2_ends_unique
Adding axiom: None_ont_t_3_occurrence_bounds
Adding axiom: None_ont_t_4_occurrence_sort_constraints
Adding axiom: None_ont_t_5_unique_activity_occurrence
Adding axiom: None_ont_t_6_occurrence_has_activity
Adding axiom: None_ont_t_7_participation_t_sort_constraints
Adding axiom: None_ont_t_8_participation_time_to_generic
Adding axiom: None_ont_t_9_participation_sort_constraints
Adding axiom: None_ont_t_10_participation_temporal_constraint
Adding axiom: None_ont_t_11_object_temporal_existence
Adding axiom: None_ont_t_12_occurrence_temporal_extent
Adding axiom: None_map_t_0_occurrence_start_end_event_mapping
Adding axiom: None_map_t_1_occurrence_participation_mapping
Adding axiom: None_map_t_2_case_process_occurrence_mapping


In [10]:
psl_core.axioms

[{'name': 'None_ont_t_0_type_disjointness',
  'description': 'Activities, occurrences, and objects are different things.',
  'expr': '(assert (! (forall ((x Ind)) (and (=> (activity x) (not (or (activity_occurrence x) (object_ x)))) (=> (activity_occurrence x) (not (or (object_ x) (activity x)))) (=> (object_ x) (not (or (activity_occurrence x) (activity x)))))) :named type_disjointness :description "Activities, occurrences, and objects are different things."))'},
 {'name': 'None_ont_t_1_begin_unique',
  'description': 'Start points are unique.',
  'expr': '(assert (! (forall ((x Ind) (t1 Real) (t2 Real)) (=> (and (= (begin_of x) t1) (= (begin_of x) t2)) (= t1 t2))) :named begin_unique :description "Start points are unique."))'},
 {'name': 'None_ont_t_2_ends_unique',
  'description': 'End points are unique.',
  'expr': '(assert (! (forall ((x Ind) (t1 Real) (t2 Real)) (=> (and (= (end_of x) t1) (= (end_of x) t2)) (= t1 t2))) :named ends_unique :description "End points are unique."))'},

In [11]:
z3_facts = []
input_data = z3_data
for i, fact in enumerate(input_data):
    prefix = f'mapped_a_{i}_'
    expr = fact
    name = prefix + f'{expr}'
    z3_facts.append({name : expr})

In [12]:
z3_facts[:5]

[{'mapped_a_0_Event(E_3)': <opal.logic.z3.z3_literal.Z3Literal at 0x7aa46b9a7fd0>},
 {'mapped_a_1_hasResource(E_0, user_1)': <opal.logic.z3.z3_literal.Z3Literal at 0x7aa46b9a7f40>},
 {'mapped_a_2_hasRecordedTime(E_3) == 1451640000': <opal.logic.z3.z3_literal.Z3Literal at 0x7aa46b9a7040>},
 {'mapped_a_3_hasLifecycleTransition(E_2, T_complete)': <opal.logic.z3.z3_literal.Z3Literal at 0x7aa46b9a7f70>},
 {'mapped_a_4_hasLifecycleTransition(E_3, T_start)': <opal.logic.z3.z3_literal.Z3Literal at 0x7aa46b9a7fa0>}]

In [13]:
module = PSL_OCC_PREC
r = Reasoner()
r.load_ontology(module)
r.load_data_declarations(mapper._z3_env)
r.load_facts(z3_facts)
r.add_relation_closure_axiom('hasCase')
r.add_relation_closure_axiom('Event')
r.add_relation_closure_axiom('Case')
#r.add_relation_closure_axiom('Transition')
#r.add_relation_closure_axiom('hasProcess')
r.add_unique_name_axioms('Event')
r.add_unique_name_axioms('Case')
s = r.check()

Loading assertion: Reference Taxonomy_ont_t_0_disjoint_classes : (assert (! (forall ((x Ind)) (and (=> (Event x) (not (or (Transition x) (Activity x) (Case x) (Resource x)))) (=> (Transition x) (not (or (Activity x) (Case x) (Resource x)))) (=> (Activity x) (not (or (Case x) (Resource x)))) (=> (Case x) (not (Resource x))))) :named disjoint_classes :description "Events, transitions, activities, cases, and resources are disjoint."))
Loading assertion: Reference Taxonomy_ont_t_1_event_order_SIMPLE : (assert (! (forall ((e1 Ind) (e2 Ind) (c Ind)) (= (and (Event e1) (Event e2) (Case c) (hasCase e1 c) (hasCase e2 c) (< (hasRecordedTime e1) (hasRecordedTime e2))) (event_earlier e1 e2 c))) :named event_order_SIMPLE))
Loading assertion: Reference Taxonomy_ont_t_2_infer_next_event : (assert (! (forall ((e1 Ind) (e2 Ind) (c Ind)) (= (and (event_earlier e1 e2 c) (not (exists ((e3 Ind) (t3 Real)) (and (Event e3) (not (= e3 e1)) (not (= e3 e2)) (hasCase e3 c) (< (hasRecordedTime e3) (hasRecordedTim

In [14]:
# Since the data is consistent with the ontology, we have a satisfying interpretation and can extract a model
m = s.model()

# write model output to txt
with open('model_output.txt', 'w') as f:
    for d in m:
        f.write(f'{d} = {m[d]}\n')

In [15]:
for d in m:
    if d in r.functions.values():
        arity = m[d].arity()
        if arity > 1:
            print(f'{d} : {m[d]}')

occ_covers : [else ->
 And(k!4456(Var(0)) == Ind!val!29,
     k!4456(Var(1)) == Ind!val!28)]
exists_at : [(Ind!val!19, 1451725200) -> True,
 (Ind!val!19, 1451724600) -> True,
 (Ind!val!18, 1451639700) -> True,
 (Ind!val!18, 1451638800) -> True,
 (Ind!val!17, 1451640900) -> True,
 (Ind!val!17, 1451640000) -> True,
 (Ind!val!36, 1451639400) -> True,
 (Ind!val!36, 1451638800) -> True,
 else ->
 Not(Or(Not(object_(Var(0))),
        Not(Var(1) + -1*end_of(Var(0)) <= 0),
        Not(Var(1) + -1*begin_of(Var(0)) >= 0)))]
participates_in : [else ->
 Or(And(k!4456(Var(0)) == Ind!val!19,
        k!4456(Var(1)) == Ind!val!24,
        Not(Var(2) == 32059199),
        Not(Var(2) == 2903623203/2),
        Not(Var(2) == 32059200),
        Not(Var(2) == 1451638800),
        Not(Var(2) == 1451639700),
        Not(Var(2) == 1451710954),
        Var(2) == 1451724600),
    And(k!4456(Var(0)) == Ind!val!18,
        k!4456(Var(1)) == Ind!val!29,
        Not(Var(2) == 32059199),
        Not(Var(2) == 2903623

In [16]:
# get the individuals loaded into the reasoner
inds = set(r.decls.values()).difference(set(r.functions.values()))
for ind in inds:
    print(ind, m[ind])
# get the universe of individuals (includes inferred individuals)
universe = m.get_universe(r.sorts['Ind'])
universe_size = len(universe)
print("Universe size:", universe_size)
print('individual size:', len(inds))

# find difference between the two sets of individuals
ind_set = set()
for ind in inds:
    ind_set.add(m[ind])
print('mapped individual size:', len(ind_set))
diff = set(universe).difference(ind_set)
print('difference size:', len(diff))
diff

E_8 Ind!val!0
A_activity_A Ind!val!38
E_3 Ind!val!7
E_0 Ind!val!1
E_7 Ind!val!4
E_1 Ind!val!8
user_0 Ind!val!19
T_complete Ind!val!16
A_activity_C Ind!val!22
P1 Ind!val!21
E_5 Ind!val!2
C_case_0 Ind!val!12
user_1 Ind!val!18
E_10 Ind!val!9
C_case_2 Ind!val!13
E_4 Ind!val!11
A_activity_B Ind!val!25
E_6 Ind!val!3
E_11 Ind!val!10
T_start Ind!val!15
C_case_1 Ind!val!14
E_2 Ind!val!6
E_9 Ind!val!5
A_activity_D Ind!val!20
user_2 Ind!val!17
Universe size: 198
individual size: 25
mapped individual size: 25
difference size: 173


{Ind!val!100,
 Ind!val!101,
 Ind!val!102,
 Ind!val!103,
 Ind!val!104,
 Ind!val!105,
 Ind!val!106,
 Ind!val!107,
 Ind!val!108,
 Ind!val!109,
 Ind!val!110,
 Ind!val!111,
 Ind!val!112,
 Ind!val!113,
 Ind!val!114,
 Ind!val!115,
 Ind!val!116,
 Ind!val!117,
 Ind!val!118,
 Ind!val!119,
 Ind!val!120,
 Ind!val!121,
 Ind!val!122,
 Ind!val!123,
 Ind!val!124,
 Ind!val!125,
 Ind!val!126,
 Ind!val!127,
 Ind!val!128,
 Ind!val!129,
 Ind!val!130,
 Ind!val!131,
 Ind!val!132,
 Ind!val!133,
 Ind!val!134,
 Ind!val!135,
 Ind!val!136,
 Ind!val!137,
 Ind!val!138,
 Ind!val!139,
 Ind!val!140,
 Ind!val!141,
 Ind!val!142,
 Ind!val!143,
 Ind!val!144,
 Ind!val!145,
 Ind!val!146,
 Ind!val!147,
 Ind!val!148,
 Ind!val!149,
 Ind!val!150,
 Ind!val!151,
 Ind!val!152,
 Ind!val!153,
 Ind!val!154,
 Ind!val!155,
 Ind!val!156,
 Ind!val!157,
 Ind!val!158,
 Ind!val!159,
 Ind!val!160,
 Ind!val!161,
 Ind!val!162,
 Ind!val!163,
 Ind!val!164,
 Ind!val!165,
 Ind!val!166,
 Ind!val!167,
 Ind!val!168,
 Ind!val!169,
 Ind!val!170,
 Ind!v

In [17]:
model_interp = {i : m[i] for i in inds}

In [18]:
model_interp

{E_8: Ind!val!0,
 A_activity_A: Ind!val!38,
 E_3: Ind!val!7,
 E_0: Ind!val!1,
 E_7: Ind!val!4,
 E_1: Ind!val!8,
 user_0: Ind!val!19,
 T_complete: Ind!val!16,
 A_activity_C: Ind!val!22,
 P1: Ind!val!21,
 E_5: Ind!val!2,
 C_case_0: Ind!val!12,
 user_1: Ind!val!18,
 E_10: Ind!val!9,
 C_case_2: Ind!val!13,
 E_4: Ind!val!11,
 A_activity_B: Ind!val!25,
 E_6: Ind!val!3,
 E_11: Ind!val!10,
 T_start: Ind!val!15,
 C_case_1: Ind!val!14,
 E_2: Ind!val!6,
 E_9: Ind!val!5,
 A_activity_D: Ind!val!20,
 user_2: Ind!val!17}

In [19]:
m = r.model
m[r.decls['hasLifecycleTransition']]

[else ->
 And(Not(And(k!4456(Var(0)) == Ind!val!11,
             Var(1) == Ind!val!15)),
     Not(And(k!4456(Var(0)) == Ind!val!10,
             Var(1) == Ind!val!15)),
     Not(And(k!4456(Var(0)) == Ind!val!1,
             Var(1) == Ind!val!16)),
     Not(And(k!4456(Var(0)) == Ind!val!2,
             Var(1) == Ind!val!16)),
     Not(And(k!4456(Var(0)) == Ind!val!6,
             Var(1) == Ind!val!15)),
     Not(And(k!4456(Var(0)) == Ind!val!7,
             Var(1) == Ind!val!16)))]

In [20]:
m[r.decls['occurrence_of']]

[else ->
 Or(And(k!4456(Var(0)) == Ind!val!2,
        k!4456(Var(1)) == Ind!val!31),
    And(k!4456(Var(0)) == Ind!val!14,
        k!4456(Var(1)) == Ind!val!21),
    And(k!4456(Var(0)) == Ind!val!12,
        k!4456(Var(1)) == Ind!val!21),
    And(k!4456(Var(0)) == Ind!val!29,
        k!4456(Var(1)) == Ind!val!38),
    And(k!4456(Var(0)) == Ind!val!28,
        k!4456(Var(1)) == Ind!val!22),
    And(k!4456(Var(0)) == Ind!val!24,
        k!4456(Var(1)) == Ind!val!25),
    And(k!4456(Var(0)) == Ind!val!13,
        k!4456(Var(1)) == Ind!val!21))]

In [21]:
m[r.decls['activity']]

[else ->
 Or(k!4456(Var(0)) == Ind!val!38,
    k!4456(Var(0)) == Ind!val!21,
    k!4456(Var(0)) == Ind!val!31,
    k!4456(Var(0)) == Ind!val!25,
    k!4456(Var(0)) == Ind!val!22)]

In [22]:
m[r.decls['activity_occurrence']]

[else ->
 Or(k!4456(Var(0)) == Ind!val!13,
    k!4456(Var(0)) == Ind!val!28,
    k!4456(Var(0)) == Ind!val!12,
    k!4456(Var(0)) == Ind!val!29,
    k!4456(Var(0)) == Ind!val!2,
    k!4456(Var(0)) == Ind!val!14,
    k!4456(Var(0)) == Ind!val!24)]

In [23]:
m[r.decls['ev_occ']]

[else ->
 If(And(k!4456(Var(0)) == Ind!val!7,
        k!4456(Var(1)) == Ind!val!53),
    Ind!val!196,
    If(And(k!4456(Var(0)) == Ind!val!7,
           k!4456(Var(1)) == Ind!val!161),
       Ind!val!195,
       If(And(k!4456(Var(0)) == Ind!val!7,
              k!4456(Var(1)) == Ind!val!177),
          Ind!val!194,
          If(And(k!4456(Var(0)) == Ind!val!7,
                 k!4456(Var(1)) == Ind!val!5),
             Ind!val!193,
             If(And(k!4456(Var(0)) == Ind!val!7,
                    k!4456(Var(1)) == Ind!val!113),
                Ind!val!192,
                If(And(k!4456(Var(0)) == Ind!val!7,
                       k!4456(Var(1)) == Ind!val!126),
                   Ind!val!191,
                   If(And(k!4456(Var(0)) == Ind!val!7,
                          k!4456(Var(1)) == Ind!val!171),
                      Ind!val!190,
                      If(And(k!4456(Var(0)) == Ind!val!7,
                             k!4456(Var(1)) == Ind!val!169),
                         Ind!val!189,
                         If(And(k!4456(Var(0)) == Ind!val!7,
                                k!4456(Var(1)) == Ind!val!62),
                            Ind!val!188,
                            If(And(k!4456(Var(0)) ==
                                   Ind!val!7,
                                   k!4456(Var(1)) ==
                                   Ind!val!158),
                               Ind!val!187,
                               If(And(k!4456(Var(0)) ==
                                      Ind!val!7,
                                      k!4456(Var(1)) ==
                                      Ind!val!128),
                                  Ind!val!186,
                                  If(And(k!4456(Var(0)) ==
                                        Ind!val!7,
                                        k!4456(Var(1)) ==
                                        Ind!val!174),
                                     Ind!val!185,
                                     If(And(k!4456(Var(0)) ==
                                        Ind!val!7,
                                        k!4456(Var(1)) ==
                                        Ind!val!170),
                                        Ind!val!184,
                                        If(And(k!4456(Var(0)) ==
                                        Ind!val!7,
                                        k!4456(Var(1)) ==
                                        Ind!val!176),
                                        Ind!val!183,
                                        If(And(k!4456(Var(0)) ==
                                        Ind!val!7,
                                        k!4456(Var(1)) ==
                                        Ind!val!173),
                                        Ind!val!182,
                                        If(And(k!4456(Var(0)) ==
                                        Ind!val!7,
                                        k!4456(Var(1)) ==
                                        Ind!val!164),
                                        Ind!val!181,
                                        If(And(k!4456(Var(0)) ==
                                        Ind!val!7,
                                        k!4456(Var(1)) ==
                                        Ind!val!162),
                                        Ind!val!180,
                                        If(And(k!4456(...) ==
                                        Ind!val!7,
                                        k!4456(...) ==
                                        Ind!val!139),
                                        Ind!val!179,
                                        If(And(... == ...,
                                        ... == ...),
                                        Ind!val!178,
                                        If(And(..., ...),
                                        Ind!val!177,
                                        If(..., ..., ...)))))))))))))))))))))]

In [24]:
m[r.decls['occ_precedes']]

[else ->
 And(k!4456(Var(0)) == Ind!val!29,
     k!4456(Var(1)) == Ind!val!28)]

In [25]:
m[r.decls['occ_covers']]

[else ->
 And(k!4456(Var(0)) == Ind!val!29,
     k!4456(Var(1)) == Ind!val!28)]

In [26]:
m[r.decls['comp_occ_precedes']]

[else ->
 Or(And(k!4456(Var(0)) == Ind!val!29,
        k!4456(Var(1)) == Ind!val!28,
        k!4456(Var(2)) == Ind!val!197),
    And(k!4456(Var(0)) == Ind!val!29,
        k!4456(Var(1)) == Ind!val!28,
        k!4456(Var(2)) == Ind!val!12))]

In [27]:
m[r.decls['participates']]

[else ->
 Or(And(k!4456(Var(0)) == Ind!val!17,
        k!4456(Var(1)) == Ind!val!28),
    And(k!4456(Var(0)) == Ind!val!19,
        k!4456(Var(1)) == Ind!val!24),
    And(k!4456(Var(0)) == Ind!val!18,
        k!4456(Var(1)) == Ind!val!29))]

In [28]:
m[r.decls['hand_off']]

[else ->
 Not(And(k!4456(Var(0)) == Ind!val!19,
         k!4456(Var(1)) == Ind!val!19,
         k!4456(Var(2)) == Ind!val!25))]

### Pattern Usage

In [52]:
from opal.logic.z3.z3_pattern import HAND_OFF_PATTERN, PING_PONG_PATTERN, PING_PONG_QUERY_PATTERN
ping_pong_q = PING_PONG_QUERY_PATTERN.apply()
s = r.query(ping_pong_q)

Adding query: p_Hand-off_ : 
(assert (! (forall ((r1 Ind) (r2 Ind) (o1 Ind) (o2 Ind) (c Ind))
    (= (and
            (next_subocc o1 o2 c)
            (participates r1 o1)
            (participates r2 o2)
            (distinct r1 r2)
        )
    (hand_off r1 r2 o1 o2 c))
)))
Adding query: p_Ping-pong_ : 
(assert (! (forall ((c Ind))
    (= (exists ((e1 Ind) (e2 Ind) (e3 Ind) (e4 Ind) (r1 Ind) (r2 Ind) (r3 Ind) (r4 Ind))
            (and
                (hand_off r1 r2 e1 e2 c)
                (hand_off r3 r4 e3 e4 c)
                (distinct e1 e3)
                (distinct e2 e4)
            ))
        (ping_pong c)
    )
)))
Adding query: p_Ping-pong Query_ : 
(assert (! (not (exists ((c Ind))
    (ping_pong c))
)))


In [53]:
r.add_relation_closure_axiom('activity_occurrence')

No instances found for relation activity_occurrence. Skipping closure.


In [54]:
s = r.check()

In [55]:
m = s.model()

In [ ]:
m[r.decls['min_precedes']]

[else -> False]

# Mapping BPIC 2017 data

In [29]:
processor = LogProcessor(log_path=bpic2017_file, batch_size=100)
event_data = processor.load_log(downsample_rate=0.05)

/home/vscode/.local/lib/python3.9/site-packages/pm4py/utils.py:800: UserWarning: Install the optional requirement `rustxes` to import/export files faster.
  warnings.warn("Install the optional requirement `rustxes` to import/export files faster.")
/home/vscode/.local/lib/python3.9/site-packages/pm4py/util/dt_parsing/parser.py:82: UserWarning: ISO8601 strings are not fully supported with strpfromiso for Python versions below 3.11
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
parsing log, completed traces :: 100%|██████████| 31509/31509 [01:00<00:00, 517.39it/s]
/workspaces/ProMean Lib/opal/data/processor.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in

In [30]:
mapper = LogMapper(data=event_data, output_encoding='z3')
bpi13_z3_data = mapper.get_mapped_data()

Building mappings for logs...


  0%|          | 0/16 [00:00<?, ?it/s]2025-11-25 22:00:33,729 | INFO: Translating YARRRML mapping to [R2]RML
INFO | 2025-11-25 22:00:33,729 | Translating YARRRML mapping to [R2]RML
2025-11-25 22:00:33,731 | INFO: RML content is created!
INFO | 2025-11-25 22:00:33,731 | RML content is created!
2025-11-25 22:00:33,740 | INFO: Mapping has been syntactically validated.
INFO | 2025-11-25 22:00:33,740 | Mapping has been syntactically validated.
2025-11-25 22:00:33,742 | INFO: Translation has finished successfully.
INFO | 2025-11-25 22:00:33,742 | Translation has finished successfully.
2025-11-25 22:00:33,753 | INFO: Translating YARRRML mapping to [R2]RML
INFO | 2025-11-25 22:00:33,753 | Translating YARRRML mapping to [R2]RML
2025-11-25 22:00:33,755 | INFO: RML content is created!
INFO | 2025-11-25 22:00:33,755 | RML content is created!
2025-11-25 22:00:33,764 | INFO: Mapping has been syntactically validated.
INFO | 2025-11-25 22:00:33,764 | Mapping has been syntactically validated.
2025-11-2

Mappings built.
Generating knowledge graph...


INFO | 2025-11-25 22:00:34,601 | 11 mapping rules retrieved.
INFO | 2025-11-25 22:00:34,616 | Mapping partition with 11 groups generated.
INFO | 2025-11-25 22:00:34,617 | Maximum number of rules within mapping group: 1.
INFO | 2025-11-25 22:00:34,618 | Mappings processed in 0.576 seconds.
INFO | 2025-11-25 22:00:36,081 | Number of triples generated in total: 23556.


Knowledge graph generated.
Knowledge graph 1/16 generated.
Generating knowledge graph...


...

Knowledge graph generated.
Knowledge graph 14/16 generated.
Generating knowledge graph...


INFO | 2025-11-25 22:01:14,976 | 11 mapping rules retrieved.
INFO | 2025-11-25 22:01:14,998 | Mapping partition with 11 groups generated.
INFO | 2025-11-25 22:01:15,000 | Maximum number of rules within mapping group: 1.
INFO | 2025-11-25 22:01:15,001 | Mappings processed in 0.606 seconds.
INFO | 2025-11-25 22:01:16,670 | Number of triples generated in total: 22542.


Knowledge graph generated.
Knowledge graph 15/16 generated.
Generating knowledge graph...


INFO | 2025-11-25 22:01:17,774 | 11 mapping rules retrieved.
INFO | 2025-11-25 22:01:17,787 | Mapping partition with 11 groups generated.
INFO | 2025-11-25 22:01:17,788 | Maximum number of rules within mapping group: 1.
INFO | 2025-11-25 22:01:17,788 | Mappings processed in 0.538 seconds.
INFO | 2025-11-25 22:01:19,286 | Number of triples generated in total: 16119.


Knowledge graph generated.
Knowledge graph 16/16 generated.
Z3 literal set (1/16) generated.
Z3 literal set (2/16) generated.
Z3 literal set (3/16) generated.
Z3 literal set (4/16) generated.
Z3 literal set (5/16) generated.
Z3 literal set (6/16) generated.
Z3 literal set (7/16) generated.
Z3 literal set (8/16) generated.
Z3 literal set (9/16) generated.
Z3 literal set (10/16) generated.
Z3 literal set (11/16) generated.
Z3 literal set (12/16) generated.
Z3 literal set (13/16) generated.
Z3 literal set (14/16) generated.
Z3 literal set (15/16) generated.
Z3 literal set (16/16) generated.


In [32]:
len(bpi13_z3_data)

368183

In [ ]:
len(mapper.data.data)

59801

In [ ]:
str(z3_data[5008])

'hasResource(E_1174728, R_User_29)'